<a href="https://colab.research.google.com/github/RichardFrZ/FiltroAcoesViaveis/blob/main/Filtro_de_a%C3%A7%C3%B5es_vi%C3%A1veis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title Importar bibliotecas
!pip install investpy
import yfinance as yf
import pandas as pd
from datetime import datetime
from gspread_dataframe import set_with_dataframe as s
import pandas as pd
import gspread
from google.colab import auth
auth.authenticate_user()
import investpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 15.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 14.3 MB/s eta 0:00:00
  Created wheel for investpy: filename=investpy-1.0.8-py3-none-any.whl size=4481565 sha256=cddd9bed505bc9953ce5826165ee7663f55183ae9596f82d358dbe6bc51a74de
  Stored in directory: /root/.cache/pip/wheels/db/f4/ae/980b93b0257620bc8e4afc98a854a7a746a33eb8335fd07906
Successfully built investpy


In [ ]:
# @title Buscar os tickers(códigos) dos ativos Brasileiros

# Importa a lista de ações disponíveis no Brasil usando a biblioteca investpy.
lista = investpy.get_stocks(country='Brazil')

# Cria uma lista com os símbolos das ações.
symbol = lista['symbol'].tolist()


In [ ]:
# @title Configurar os tickers e o período dos dados (geral)

# Adiciona a extensão .SA nos ativos para a pesquisa.
tickers = symbol
tickers = [tickers + ".SA" for tickers in tickers]

# Define a data final como o ultimo dia do ano anterior.
data_final = datetime(datetime.now().year - 1, 12, 31) - pd.DateOffset(years= 0)

# Define a data inicial como 8 anos antes da data final.
data_inicial = data_final - pd.DateOffset(years= 8)


In [ ]:
# @title Baixar dados dos ativos

# Baixa os dados de dividendos.
dados = yf.download(tickers=tickers, start=data_inicial, end=data_final, actions=True)["Dividends"]

[*********************100%***********************]  747 of 747 completed
ERROR:yfinance:
169 Failed downloads:
ERROR:yfinance:['CARD3.SA', 'CCPR3.SA', 'WTSP11B.SA', 'GNDI3.SA', 'OMGE3.SA', 'BKBR3.SA', 'BMGB11.SA', 'VLJS11.SA', 'DMMO11.SA', 'JBDU3.SA', 'BEEF11.SA', 'NATU3.SA', 'CELP7.SA', 'ENMA3B.SA', 'CEPE5.SA', 'ELDO11B.SA', 'MEND5.SA', 'XTED11.SA', 'IDVL9.SA', 'BTOW3.SA', 'PRTS11.SA', 'HGTX3.SA', 'CAMB10.SA', 'LAME4.SA', 'CRDE3.SA', 'TORM13.SA', 'LINX3.SA', 'LOGN12.SA', 'WIZS3.SA', 'CESP3.SA', 'TIET2.SA', 'HBTT11.SA', 'GESE11B.SA', 'ATCR11.SA', 'MEND6.SA', 'FEXC11.SA', 'SEDU3.SA', 'TIMP3.SA', 'TIET4.SA', 'LLIS3.SA', 'CXCE11B.SA', 'DMMO3T.SA', 'VVAR3.SA', 'OUCY11.SA', 'BIDI4T.SA', 'FVBI11.SA', 'SHDP11B.SA', 'BBFI11B.SA', 'WSON33.SA', 'BIDI3.SA', 'DRIT11B.SA', 'TIET11.SA', 'PRSN11B.SA', 'LIQO3.SA', 'TWTR34.SA', 'IDNT3.SA', 'TCPF11.SA', 'GBIO33.SA', 'IDVL3.SA', 'TASA13.SA', 'LAME3.SA', 'EEEL3.SA', 'FAMB11B.SA', 'VIVT4.SA', 'TIET3.SA', 'PNVL4.SA', 'ALSO3.SA', 'BBRK3.SA', 'TOYB3.SA', 'TRX

In [ ]:
# @title Agrupa e filtra ativos que não distribuiram dividendos nos ultimos 5 anos

# Agrupa o DataFrame por ano e soma os dividendos para cada ano.
dados_agrupados = dados.groupby(dados.index.year).sum()

# Seleciona os últimos 5 anos de dados.
dados_agrupados = dados_agrupados.tail(5)

# Identifica colunas (empresas) que possuem valores de dividendos distribuídos iguais a zero.
df = dados_agrupados.columns[dados_agrupados.eq(0).any()]

# Remove essas empresas do DataFrame 'dados_agrupados'.
dados_agrupados = dados_agrupados.drop(columns=df)

In [ ]:
# @title Cálculo preço teto

# Calcula a média de dividendos para cada empresa restante.
dados_agrupados = dados_agrupados.mean()

# Calcula o preco teto dividindo o dividendo médio por 0.06 (6%) e arredondando para 2 casas decimais.
preco_teto = round(dados_agrupados/0.06,2)

In [ ]:
# @title Coletando o Preço Atual dos ativos

#Busca os códigos dos ativos que foi possível calcular o preço teto.
tickers_selecionados = dados_agrupados.index.tolist()

#Busca os preços atuais dos ativos.
preco_atual = yf.download(tickers = tickers_selecionados, period='1mo')['Close']

#Seleciona apenas a última cotação.
preco_atual = round(preco_atual.iloc[-1],2)

[*********************100%***********************]  276 of 276 completed


In [ ]:
# @title Cálculo da liquidez e cria filtro dos ativos

#Busca a liquidez dos ativos em um período de 3 meses.
dados_volume = yf.download(tickers = tickers_selecionados, period='3mo')['Volume']

#Calcula a média de liquidez dos ativos.
dados_volume = round(dados_volume.mean())

#Seleciona apenas os ativos com liquidez maior que 1 Milhão.
liquidez = dados_volume[dados_volume > 1000000]

[*********************100%***********************]  276 of 276 completed


In [ ]:
# @title Nomeando colunas

# Define o nome do índice como ATIVO.
for df in [preco_teto, preco_atual, liquidez]:
    df.index.name = "ATIVO"

# Criando um DataFrame chamado 'df_preco_teto'.
df_preco_atual = pd.DataFrame(preco_atual)

# Criando um DataFrame chamado 'df_liquidez'.
df_liquidez = pd.DataFrame(liquidez)

# Criando um DataFrame chamado 'df_preco_teto'.
df_preco_teto = pd.DataFrame(preco_teto)

# Define o nome da coluna como "PREÇO TETO".
df_preco_teto.columns = ["PREÇO TETO"]

# Define o nome da coluna como "PREÇO ATUAL".
df_preco_atual.columns = ["PREÇO ATUAL"]

# Define o nome da coluna como "LIQUIDEZ".
df_liquidez.columns = ["LIQUIDEZ"]

# Remove os últimos 3 caracteres (".SA") de cada ativo dos DataFrames 'df_preco_teto, df_preco_atual, df_liquidez'.
for df in [df_preco_teto, df_preco_atual, df_liquidez]:
    df.index = df.index.str[:-3]


In [ ]:
# @title Criação do DataFrame de ações viaveis

# Combina os DataFrames Preço Teto e Preço Atual.
df_ativos = pd.merge(df_preco_teto, df_preco_atual, left_index=True, right_index=True)

# Combina os DataFrames df_ativos e Liquidez filtrando para apenas ativos com Liquidez.
df_ativos = pd.merge(df_ativos, df_liquidez, left_index=True, right_index=True)

# Filtra o DataFrame para ativos com preço atual menor que o preço teto.
df_ativos = df_ativos[df_ativos["PREÇO ATUAL"] < df_ativos["PREÇO TETO"]]


In [ ]:
# @title Visualização do DataFrame

df_ativos

,PREÇO TETO,PREÇO ATUAL,LIQUIDEZ
ATIVO,,,
BBDC3,14.97,14.17,7143200.0
BBDC4,18.32,15.67,37715942.0
BBSE3,40.57,36.41,4433808.0
BRAP4,66.32,19.30,2916959.0
BRSR6,16.32,12.58,1404195.0
CMIG4,12.85,11.55,12960136.0
CPFE3,40.37,33.04,1493676.0
CSMG3,25.97,23.43,1475011.0
CSNA3,22.81,11.91,6452836.0


### **Envio para o Google Sheets**

In [ ]:
# @title Necessário conectar conta Google
import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

for ticker in df_ativos.index:
  # Verifica se o preço teto em df_dados é igual ao preço teto em df_preco_atual
  if df_ativos.loc[ticker, "PREÇO ATUAL"] != df_preco_atual.loc[ticker, "PREÇO ATUAL"]:
    print(f"O preço teto de {ticker} em df_dados é diferente do preço teto em df_preco_atual.")

for ticker in df_ativos.index:
  # Verifica se o preço teto em df_dados é igual ao preço teto em df_preco_teto
  if df_ativos.loc[ticker, "PREÇO TETO"] != df_preco_teto.loc[ticker, "PREÇO TETO"]:
    print(f"O preço teto de {ticker} em df_dados é diferente do preço teto em df_preco_teto.")

for ticker in df_ativos.index:
  # Verifica se a liquidez em df_dados é igual a liquidez em df_liquidez
  if df_ativos.loc[ticker, "LIQUIDEZ"] != df_liquidez.loc[ticker, "LIQUIDEZ"]:
    print(f"A liquidez de {ticker} em df_dados é diferente da liquidez em df_liquidez.")

cria_file = False

# Cria e define a planilha que irá a saída dos dados
if cria_file:
  gc.create('Ativos Viáveis')

worksheet = gc.open('Ativos Viáveis').worksheet('Página1')

# Limpa as colunas A até B na planilha especificada.
worksheet.batch_clear(["A:D"])

# Exporta o DataFrame 'df_preco_teto' para a planilha do Google Sheets, incluindo o índice.
s(worksheet, df_ativos, include_index= 'true')